In [72]:
from dotenv import load_dotenv

from sentence_transformers import SentenceTransformer

from openai import OpenAI

from elasticsearch import Elasticsearch


In [302]:
#!pip install pandas

# 

In [2]:
load_dotenv()

True

# 

# 

First of all we will load de embeddings model, the vector db generated in the ingestion step to continue with the app.

Once it's finished, we'll generate the .py associated leaving only the code needed for the production version of the app.

In [70]:
# Initialize the embeddings model
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize the Elasticsearch client
es_client = Elasticsearch("http://localhost:9200")

c:\Users\TALIGENT\Desktop\Proyectos\LLM-Zoomcamp\MessiXpert-RAG\messiXpert-RAG-venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [71]:
# ElasticSearch Index names
index_name_cosine = "messixpert_cosine"
index_name_dot_product = "messixpert_dot_product"

# 

# 

# 

# RAG Flow

In [44]:
def text_search(user_query, es_client, index):
    query = {
        "size": 5,  
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "content": user_query  # Search in the content field
                        }
                    }
                ]
            }
        }
    }

    results = es_client.search(index=index, body=query)
    
    return results["hits"]["hits"]

In [59]:
def knn_search(user_query, es_client, index):

    user_query = embeddings_model.encode(user_query)

    query = {
        "k": 5,  
        "field": "content_embeddings",
        "query_vector": user_query
    }

    results = es_client.search(index=index, knn=query)

    return results["hits"]["hits"]

In [54]:
def get_answers_content(answers):

    retrieved_texts = [hit["_source"]["content"] for hit in answers]

    return retrieved_texts
    

In [56]:
def build_prompt(query, search_results_text_list):
    
    prompt_template = """
You are an expert biographer on the life and career of Lionel Messi, with deep knowledge of his entire history and statistics. 
Your task is to answer users questions based solely on the context provided to you. 
Answer respectfully and in a warm way, as if you are an assistant.
Answer in the same language that you are asked, if you are asked in english then answer in english, but if you are asked in spanish then answer in spanish.
Use only the data from the context to answer the question. 
If you cannot answer the question with the provided information, respond: “I’m sorry, but I don’t have enough information to answer that. Is there anything else I can help you with?”

CONTEXT: {context}

QUESTION: 
{question}
""".strip()

    context = ""
    
    for text in search_results_text_list:
        context = context + f"{text}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [57]:
def calculate_cost(response):

    input_tokens = response.usage.prompt_tokens
    output_tokens = response.usage.completion_tokens

    input_tokens_cost_per_1k = 0.00015
    output_tokens_cost_per_1k = 0.0006

    input_tokens_cost = input_tokens_cost_per_1k * (input_tokens / 1000)
    output_tokens_cost = output_tokens_cost_per_1k * (output_tokens / 1000)
    total_cost = input_tokens_cost + output_tokens_cost

    print("------------------------------------")
    print(f"Input Tokens: {input_tokens}       Cost: ${input_tokens_cost:.8f}")
    print(f"Completion Tokens: {output_tokens}       Cost: ${output_tokens_cost:.8f}")
    print(f"Total Cost: ${total_cost:.8f}")
    print("------------------------------------")

    return total_cost

In [58]:
def llm_generate_answer(prompt, open_ai_client):
    response = open_ai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    calculate_cost(response)

    return response.choices[0].message.content

In [64]:
def generate_answer(question, search_function, es_client, index_name, open_ai_client):

    top_k_chunks = search_function(user_query=question, es_client=es_client, index=index_name)

    answers = get_answers_content(top_k_chunks)
    
    builded_prompt = build_prompt(query=question, search_results_text_list=answers)
    
    answer = llm_generate_answer(builded_prompt, open_ai_client=open_ai_client)

    return answer

In [63]:
OpenAI_client = OpenAI()

In [65]:
question = "Cuántos hermanos tiene Messi?"

In [68]:
answer_knn = generate_answer(question=question,
                             search_function=knn_search, 
                             es_client=es_client, 
                             index_name=index_name_cosine,
                             open_ai_client=OpenAI_client)

print(answer_knn)

------------------------------------
Input Tokens: 1139       Cost: $0.00017085
Completion Tokens: 33       Cost: $0.00001980
Total Cost: $0.00019065
------------------------------------
Lionel Messi tiene tres hermanos: dos mayores, Rodrigo y Matías, y una hermana menor, María Sol. ¿Hay algo más en lo que pueda ayudarte?


In [69]:
answer_text = generate_answer(question=question,
                             search_function=text_search, 
                             es_client=es_client, 
                             index_name=index_name_cosine,
                             open_ai_client=OpenAI_client)

print(answer_text)

------------------------------------
Input Tokens: 2157       Cost: $0.00032355
Completion Tokens: 37       Cost: $0.00002220
Total Cost: $0.00034575
------------------------------------
Lionel Messi tiene tres hermanos: dos hermanos mayores, Rodrigo y Matías, y una hermana menor, María Sol. Si necesitas saber algo más, estaré encantado de ayudarte.
